In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Create subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [5]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.2", temperature=0.0)

# Create subagents

subagent_1 = create_agent(
    model=model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model=model,
    tools=[square]
)

## Calling subagents

In [6]:
from langchain.messages import HumanMessage
from langchain.agents import create_agent

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke(
        {"messages": [HumanMessage(content=f"Calculate the square root of {x}")]}
    )
    return response["messages"][-1].content


@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke(
        {"messages": [HumanMessage(content=f"Calculate the square of {x}")]}
    )
    return response["messages"][-1].content


# Create the main supervisor agent

supervisor_agent = create_agent(
    model=model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.",
)

## Test

In [7]:
question = "What is the square root of 456?"

response = supervisor_agent.invoke({"messages": [HumanMessage(content=question)]})

In [8]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='7c51add1-84c8-4df6-9111-de8ac889f081'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-01-26T14:51:04.709273Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4163501916, 'load_duration': 3173410833, 'prompt_eval_count': 228, 'prompt_eval_duration': 532531667, 'eval_count': 20, 'eval_duration': 348506918, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019bfac9-6228-7502-b22a-d7cd3268af33-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': '1ba91bf0-11e1-4313-a9ca-e32bcdbf5225', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 228, 'output_tokens': 20, 'total_tokens': 248}),
              ToolMessage(content='The square root of 456.0 is approximately 21.3542.', name='call_subagent_1', id='6a384196-397f-4a20-b8

In [9]:
question = "What is the square of 12?"

response = supervisor_agent.invoke({"messages": [HumanMessage(content=question)]})

In [10]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square of 12?', additional_kwargs={}, response_metadata={}, id='6791ab8c-8a7a-4947-8909-0be4b306f772'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-01-26T14:52:04.991674Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1467943167, 'load_duration': 148305542, 'prompt_eval_count': 227, 'prompt_eval_duration': 864095166, 'eval_count': 20, 'eval_duration': 350993376, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019bfaca-583e-77f3-8347-ca1d94e17c31-0', tool_calls=[{'name': 'call_subagent_2', 'args': {'x': 12}, 'id': '8a07c5db-92c6-41c2-897c-29ba6bf3e1cc', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 227, 'output_tokens': 20, 'total_tokens': 247}),
              ToolMessage(content='The square of 12.0 is 144.0.', name='call_subagent_2', id='f18b6337-0d6a-4b9c-9e38-fd974fb583a7', tool_call_id

Trace: https://smith.langchain.com/public/6e2224bd-ead8-4cff-9a0b-6bb47f5227fb/r